In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
class socketio(object):
    @classmethod
    def emit(msg, msg1, msg2, **kw):
        print(msg1, msg2)

In [3]:
market = {'name': 'Eyelash', 'directory': "E:\\glitter\\Eyelashes"}
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
p4p = P4P(market, lid, lpwd, socketio)

E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [5]:
start = time.time()
p4p.crawl()
end = time.time()
total = end - start
print(total)

E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
True > index: 0 50 > 3d eyelash extension > ['18.6', '18.1', '17.5', '16.0', '16.0']> >>>>>> successful end 
False > index: 1 50 > 3d mink eyelashes > ['32.7', '32.2', '31.9', '30.7', '25.1']> >>>>>> successful end 
False > index: 2 50 > 3d mink lashes private label > ['27.1', '25.7', '23.9', '18.4', '18.1']> >>>>>> successful end 
False > index: 3 50 > 3d silk eyelash > ['27.0', '19.6', '19.2', '19.1', '18.1']> >>>>>> successful end 
False > index: 4 50 > 3d silk lashes > ['23.9', '20.1', '19.3', '19.1', '19.1']> >>>>>> successful end 
False > index: 5 50 > cluster eyelash > ['12.9', '12.9', '11.7', '10.9', '10.6']> >>>>>> successful end 
False > index: 6 50 > custom eyelash box > ['24.7', '17.7', '17.6', '17.4', '17.2']> >>>>>> successful end 
False > index: 7 50 > custom eyelash packaging > ['21.5', '21.4', '21.3', '20.9', '19.7']> >>>>>> suc

In [32]:
total

75.36266899108887

In [7]:
def load_url(self):

    if 'browser' not in self:
        self.logger.info('open browser and login')
        alibaba = Alibaba(lid, lpwd, socketio, namespace, room)
        alibaba.login()
        self.browser = alibaba.browser

    self.browser.get(self.api)
    WebDriverWait(self.browser, 15).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.bp-loading-panel')))
    # css_selector = "div.keyword-manage .bp-table-main-wraper>table tbody tr:first-child"
    # tr = self.browser.find_element_by_css_selector(css_selector)
    # WebDriverWait(self.browser, 15).until(EC.staleness_of(tr))
    btn_close = self.browser.find_elements_by_css_selector('.J_follow_me_dialog_close')
    if btn_close:
        btn_close[0].click()
p4p.load_url = types.MethodType(load_url, p4p)

In [ ]:
from libs.json import JSON

In [ ]:
from pyquery import PyQuery as pq

In [ ]:
css_selector = "div.m-product-item .item-extra, div.m-product-item .brand-right-container"
WebDriverWait(p4p.browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

In [ ]:
p4p.browser.find_element_by_css_selector(css_selector)

In [ ]:
p4p.set_keywords('monitored', ['ding dong', 'chang shu'])

In [ ]:
p4p.add_keywords('monitored', '3d silk eyelash')

In [ ]:
p4p.keywords_list

In [ ]:
p4p.del_keywords('monitored', 'chang shu')

In [8]:

import types

In [ ]:
class P4P():
    
    api = 'https://www2.alibaba.com/manage_ad_keyword.htm'
    keywords_list = {}
    
    def __init__(self, market, lid, lpwd, socketio, namespace=None, room=None):
        self.market = market
        self.keywords_history = []
        self.load_keywords('recording')
        self.load_keywords('monitor')

        chrome_options_headless = webdriver.ChromeOptions()
        # chrome_options_headless.add_argument('--headless')
        chrome_options_headless.add_argument('--disable-gpu')
        chrome_options_headless.add_argument('--disable-extensions')
        chrome_options_headless.add_argument('--disable-logging')
        self.browser = webdriver.Chrome(chrome_options=chrome_options_headless)
        Alibaba.login(lid, lpwd, self.browser, socketio, namespace, room)

        self.lock = threading.RLock()

    def load_keywords(self, tp):
        if not tp:
            return
        else:
            fn = 'p4p_keywords_list_'+tp+'.json'
            root = self.market['directory'] + '_config'
            kws_list = JSON.deserialize(root, [], fn)
            if kws_list is None:
                kws_list = []
            self.keywords_list[tp] = kws_list
            
    def set_keywords(self, tp, kws_list):
        self.keywords_list[tp] = kws_list
        fn = 'p4p_keywords_list_'+tp+'.json'
        root = self.market['directory'] + '_config'
        JSON.serialize(self.keywords_list[tp], root, [], fn)
    
    def add_keywords(self, tp, kws):
        self.keywords_list[tp].append(kws)
        fn = 'p4p_keywords_list_'+tp+'.json'
        root = self.market['directory'] + '_config'
        JSON.serialize(self.keywords_list[tp], root, [], fn)
    
    def del_keywords(self, tp, kws):
        if kws in self.keywords_list[tp]:
            self.keywords_list[tp].remove(kws)
            fn = 'p4p_keywords_list_'+tp+'.json'
            root = self.market['directory'] + '_config'
            JSON.serialize(self.keywords_list[tp], root, [], fn)
    
    def crawl_keywords(self):
        keywords = []
        with self.lock:
            self.browser.get(self.api)
            css_selector = "div.keyword-manage table tbody tr:first-child"
            tr = self.browser.find_element_by_css_selector(css_selector)
            WebDriverWait(self.browser, 15).until(EC.staleness_of(tr))
            css_selector = '.bp-table-footer .ui2-pagination-pages span.current'
            WebDriverWait(self.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))

            while True:
                html = self.browser.find_element_by_css_selector("div.keyword-manage .bp-table-main-wraper>table tbody").get_attribute('outerHTML')
                tbody = pq(html)
                trs = tbody.find('tr')

                for tr in trs:
                    kws = {}
                    kws['id'] = pq(tr).find('td:first-child input').val()
                    kws['status'] = pq(tr).find('td.bp-cell-status .bp-dropdown-main i').attr('class').split('-').pop()
                    kws['kws'] = pq(tr).find('td.bp-cell-left').text().strip()
                    kws['group'] = pq(tr).find('td[data-role="table-col-tag"]').text().strip()
                    kws['my_price'] = pq(tr).find('td:nth-child(5) a').text().strip()
                    kws['average_price'] = pq(tr).find('td:nth-child(6)').text().strip()
                    string = pq(tr).find('span.qs-star-wrap i').attr('class')
                    kws['match_level'] = re.search('qsstar-(\d+)',string).group(1)
                    string = pq(tr).find('.bp-icon-progress-orange').html()
                    kws['search_count'] = re.search(':(\d+%)',string).group(1)
                    string = pq(tr).find('.bp-icon-progress-blue').html()
                    kws['buy_count'] = re.search(':(\d+%)',string).group(1)
                    keywords.append(kws)

                try:
                    btn_next = self.browser.find_element_by_css_selector('.bp-table-footer a.next')
                    css_selector = "div.keyword-manage .bp-table-main-wraper>table tbody tr:first-child"
                    tr = self.browser.find_element_by_css_selector(css_selector)
                    btn_next.click()
                    WebDriverWait(self.browser, 15).until(EC.staleness_of(tr))
                except NoSuchElementException as e:
                    break

        root = self.market['directory'] + '_config'
        fn = 'p4p_keywords_list.json'
        JSON.serialize(keywords, root, [], fn)
        return keywords

    def crawl(self):
        kws_set = set(self.keywords_list['recording'])
        keywords = []
        with self.lock:
            self.browser.get(self.api)
            # if 'alibaba.com/manage_ad_keyword.htm' not in self.browser.current_url:
            #     self.browser.get(self.api)
            # else:
            #     css_selector = '.bp-table-footer span.current'
            #     current_pn = self.browser.find_element_by_css_selector(css_selector).text
            #     if current_pn != '1':
            #         self.browser.find_element_by_css_selector('.bp-table-footer input[type="text"]').send_keys('1')
            #         css_selector = "div.keyword-manage table tbody tr:first-child"
            #         tr = self.browser.find_element_by_css_selector(css_selector)
            #         self.browser.find_element_by_css_selector('.bp-table-footer input[type="submit"]').click()
            #         WebDriverWait(self.browser, 15).until(EC.staleness_of(tr))
            
            css_selector = "div.keyword-manage table tbody tr:first-child"
            tr = self.browser.find_element_by_css_selector(css_selector)
            WebDriverWait(self.browser, 15).until(EC.staleness_of(tr))

            while True:
                table =  self.browser.find_element_by_css_selector("div.keyword-manage table")
                trs = table.find_elements_by_css_selector("tbody tr")
                for idx, tr in enumerate(trs):
                    dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
                    if kws_set and kws not in kws_set:
                        continue
                    group = tr.find_element_by_css_selector('td:nth-child(4)').text.strip()

                    btn = tr.find_element_by_css_selector('td:nth-child(5) a')
                    self.browser.implicitly_wait(1)
                    id = btn.get_attribute('data-id')
                    WebDriverWait(self.browser, 15).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.ui-mask')))
                    while True:
                        try:
                            btn.click()
                            break
                        except WebDriverException as e:
                            if 'is not clickable at point' in str(e):
                                self.browser.implicitly_wait(0.5)
                                continue
                            else:
                                raise e

                    checkbox = tr.find_element_by_css_selector('td:first-child input')
                    if checkbox.is_selected():
                        checkbox.click()
                        continue

                    sponsors = self.find_sponsors(kws)

                #     time.sleep(0.5)
                    prices = []
                    WebDriverWait(self.browser, 15).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.bp-loading-panel')))
                    while True:
                        try:
                            price_table_tbody_selector = ".sc-manage-edit-price-dialog table tbody,.sc-manage-edit-price-dialog p.util-clearfix"
                            price_table_tbody = WebDriverWait(self.browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, price_table_tbody_selector)))
#                             print(price_table_tbody.tag_name, id, kws)
                            if price_table_tbody.tag_name == 'p':
                                break

                            for btn in price_table_tbody.find_elements_by_css_selector('a'):
                                prices.append(btn.text.strip())
                            break
                        except StaleElementReferenceException:
                            self.browser.implicitly_wait(0.5)
                            continue

                    webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
                    if prices:
                        keywords.append([dt, id, kws, group, prices, sponsors])
                        
                try:
                    btn_next = self.browser.find_element_by_css_selector('.bp-table-footer a.next')
                    css_selector = "div.keyword-manage table tbody tr:first-child"
                    tr = self.browser.find_element_by_css_selector(css_selector)
                    btn_next.click()
                    WebDriverWait(self.browser, 15).until(EC.staleness_of(tr))
                except NoSuchElementException as e:
                    break

        self.save_crawling_result(keywords)
        return {'p4p_keywords_crawl_result': keywords}

    def save_crawling_result(self, keywords):
        root = self.market['directory'] + '_config'
        date_str = arrow.now().format('YYYY-MM-DD')
        fn = 'p4p_keywords_crawl_result_'+date_str+'.json.gz'
        JSON.serialize(keywords, root, [], fn, append=True)

    def turn_on(self, kws):
        with self.lock:
            pass
        pass

    def turn_off(self, kws):
        with self.lock:
            pass
        pass

    def check_and_hold_on(self, kws, position, start, end):
        with self.lock:
            pass
        pass

    def find_sponsors(self, kws):
        with self.lock:
            url = 'https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText='+re.sub(' +', '+', kws)

            if len(self.browser.window_handles) == 1:
                self.browser.execute_script("window.open()")
            self.browser.switch_to_window(self.browser.window_handles[1])

            top_sponsor = None
            sponsor_list = []

            try:
                self.browser.get(url)
                css_selector = "div.m-product-item .item-extra, div.m-product-item .brand-right-container"
                WebDriverWait(self.browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
                
                html = self.browser.page_source
                soup = BeautifulSoup(html, 'html.parser')

                divs = soup.find_all('div', class_='m-product-item')
                for idx,div in enumerate(divs):
#                     print(kws, idx)
                    company = {}
                    extra = div.find(class_='item-extra')
                    if extra:
                        a = extra.find('div', class_='stitle').a
                        company['years'] = re.search('year-num(\d+)', str(a.i)).group(1)
                        a = a.next_sibling.next_sibling
                        company['name'] = a.string.strip()
                        company['url'] = 'https:' + a['href']
                        ul = extra.find('ul', class_='record')
                    else:
                        container = div.find('div', class_='brand-right-container')
                        text = str(container.find('i', class_='year-num'))
                        company['years'] = re.search('year-num(\d+)', text).group(1)
                        a = container.find('div', class_='supplier').a
                        company['name'] = a.string.strip()
                        company['url'] = 'https:' + a['href']
                        ul = container.find('ul', class_='record-container')

                    if ul:
                        company['record'] = [re.sub('\n', '', x).strip() for x in ul.findAll(text=True) if x != '\n']

                    if div.find('span', class_='sking'):
                        top_sponsor = company
                    elif div.find('span', class_='sl'):
                        sponsor_list.append(company)
                    else:
                        break

            except Exception as e:
                print('Error: ', e)
                traceback.print_exc()
            finally:
#                 self.browser.execute_script("window.close()")
                self.browser.switch_to_window(self.browser.window_handles[0])

        return {'top_sponsor':top_sponsor, 'sponsor_list':sponsor_list}
